Q3: Implementation of Neural Networks from Scratch Using NumPy and Comparison with Sklearn (20 marks):
  
    1. Load and preprocess the MNIST Digits Dataset. (3 marks)
    2. Implement a neural network with one input layer, one hidden layer, and one output layer using NumPy. (5 marks)
    3. Train the neural network with various hyperparameters (e.g., learning rate, number of hidden nodes). (3 marks)
    4. Evaluate the performance of the neural network on the testing set. (2 marks)
    5. Implement the same neural network using sklearn and compare the results with the NumPy implementation. (4 marks)
    6. Plot the training and validation loss/accuracy curves (for both experiments). (3 marks) 



In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, classification_report
from sklearn.pipeline import Pipeline

In [2]:
# from sklearn.datasets import fetch_openml

# X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
# X["Y"] = y
# X.to_csv("mnist_784.csv",index=False,header=True)

In [3]:
mnist_784_df = pd.read_csv("mnist_784.csv")
# display(mnist_784_df.head().T)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [36]:
class NeuralNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.weights1 = np.random.rand(input_dim, hidden_dim)
        self.weights2 = np.random.rand(hidden_dim, output_dim)
        self.bias1 = np.zeros((1, hidden_dim))
        self.bias2 = np.zeros((1, output_dim))

    def forward_propagation(self, inputs):

        hidden_layer_input = np.dot(inputs, self.weights1) + self.bias1
        hidden_layer_output = relu(hidden_layer_input)
        
        output_layer_input = np.dot(hidden_layer_output, self.weights2) + self.bias2
        output_layer_output = sigmoid(output_layer_input)
        
        return hidden_layer_output, output_layer_output

    def backward_propagation(self, inputs, targets, hidden_layer_output, output_layer_output):

        d_output = 2 * (output_layer_output - targets)
        d_weights2 = np.dot(hidden_layer_output.T, d_output * sigmoid_derivative(output_layer_output))
        d_bias2 = np.sum(d_output * sigmoid_derivative(output_layer_output), axis=0, keepdims=True)
        
        d_hidden_layer = np.dot(d_output * sigmoid_derivative(output_layer_output), self.weights2.T) * relu_derivative(hidden_layer_output)
        d_weights1 = np.dot(inputs.T, d_hidden_layer)
        d_bias1 = np.sum(d_hidden_layer, axis=0, keepdims=True)
        
        return d_weights1, d_bias1, d_weights2, d_bias2

    def update_weights(self, d_weights1, d_bias1, d_weights2, d_bias2, learning_rate):
        self.weights1 -= learning_rate * d_weights1
        self.bias1 -= learning_rate * d_bias1
        self.weights2 -= learning_rate * d_weights2
        self.bias2 -= learning_rate * d_bias2

In [37]:
inputs = mnist_784_df.drop(columns="Y").values
targets = mnist_784_df["Y"].values
result = np.where(targets == 5, targets, 0)

IL_n = mnist_784_df.shape[-1]-1
n_neurons_HL = int(np.sqrt(IL_n-1))

In [41]:
# import numpy as np

# # Example 2D array
# array = np.array([[1, 10], [2, 9], [3, 8], [4, 7], [5, 6], [6, 5]])

# Normalize the array
max_values = np.max(inputs, axis=0)
min_values = np.min(inputs, axis=0)

normalized_array = ((inputs - min_values) / (max_values - min_values)).fillna(0)

print(normalized_array[0])


AttributeError: 'numpy.ndarray' object has no attribute 'fillna'

In [32]:
scaler = StandardScaler()
inputs = pd.DataFrame(scaler.fit_transform(inputs), columns=inputs)

ValueError: Index data must be 1-dimensional

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    inputs, result,
    test_size=0.20, random_state=42,
    stratify=result,shuffle=True)

In [ ]:
nn = NeuralNetwork(input_dim=IL_n, hidden_dim=n_neurons_HL, output_dim=1)


learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    hidden_layer_output, output_layer_output = nn.forward_propagation(inputs)
    d_weights1, d_bias1, d_weights2, d_bias2 = nn.backward_propagation(inputs, targets, hidden_layer_output, output_layer_output)
    nn.update_weights(d_weights1, d_bias1, d_weights2, d_bias2, learning_rate)
    
    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
        loss = np.mean(np.square(output_layer_output - targets))
        print(f"Epoch {epoch+1}, Loss: {loss}")

# Final predictions
_, final_output = nn.forward_propagation(inputs)
print("Final Predictions:")
print(final_output)


array([[0.70608995, 0.73981399, 0.84289354],
       [0.77103085, 0.4762833 , 0.45169264]])

In [ ]:
class Neuron:
    def __init__(self,value):
        self.value = value
class hidden_layer  


In [ ]:
class aHL:
    def __init__(self,mnist_784_df):

    
    def forward_prop_relu(self,input,weight):
        return np.maximum(0, input*weight)




In [14]:
X.to_csv("mnist_784.csv",index=False,header=True)

In [13]:
display(X.head().T)

,0,1,2,3,4
pixel1,0,0,0,0,0
pixel2,0,0,0,0,0
pixel3,0,0,0,0,0
pixel4,0,0,0,0,0
pixel5,0,0,0,0,0
...,...,...,...,...,...
pixel781,0,0,0,0,0
pixel782,0,0,0,0,0
pixel783,0,0,0,0,0
pixel784,0,0,0,0,0


In [3]:
df = pd.DataFrame(X, columns=[f'Pixel_{i}'for i in range(X.shape[-1])])

In [4]:
display(df.head())

,Pixel_0,Pixel_1,Pixel_2,Pixel_3,Pixel_4,Pixel_5,Pixel_6,Pixel_7,Pixel_8,Pixel_9,...,Pixel_774,Pixel_775,Pixel_776,Pixel_777,Pixel_778,Pixel_779,Pixel_780,Pixel_781,Pixel_782,Pixel_783
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
y.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']